# 03 — Change detection explained: db(t0), db(t1), ratio_db
This notebook focuses on **interpretation**:
- what db(t0) and db(t1) mean
- why we use **ratio in dB**
- how to reason about increases and decreases

It expects `sar101.py` outputs in `../outputs` (or adjust OUTDIR).


In [ ]:
from pathlib import Path
import numpy as np
import rasterio
import matplotlib.pyplot as plt

OUTDIR = Path('../outputs')
ratio_path = OUTDIR / 'ratio_db_t1_minus_t0.tif'

print('ratio_db:', ratio_path, 'exists=', ratio_path.exists())


## Load ratio_db
The script writes `ratio_db = db(t1) - db(t0)`.
This is equivalent to `10*log10(t1/t0)`.


In [ ]:
with rasterio.open(ratio_path) as ds:
    ratio_db = ds.read(1).astype('float32')
    nodata = ds.nodata
    if nodata is not None:
        ratio_db[ratio_db == nodata] = np.nan

print('ratio_db stats:', np.nanmin(ratio_db), np.nanmean(ratio_db), np.nanmax(ratio_db))


## Visualize ratio_db

In [ ]:
plt.figure(figsize=(7,6))
plt.imshow(ratio_db, vmin=-5, vmax=5)
plt.title('ratio_db = db(t1) - db(t0)')
plt.colorbar(label='dB')
plt.axis('off')
plt.show()


## Histogram + interpretation
### Interpreting sign
- **Positive** ratio_db: backscatter increased (surface became rougher / more reflective)
- **Negative** ratio_db: backscatter decreased (surface became smoother / less reflective)

### Common physical interpretations (context-dependent)
- Flooding can **decrease** VV backscatter over open areas (smoother surface) → negative ratio
- Vegetation changes can affect VH strongly (volume scattering)
- Urban/structure changes can cause strong positive changes (double bounce)

### Caveat
SAR is sensitive to geometry, moisture, and speckle — treat this as a **screening layer**, not a verdict.


In [ ]:
data = ratio_db[np.isfinite(ratio_db)]
plt.figure()
plt.hist(data, bins=200)
plt.title('ratio_db histogram')
plt.xlabel('dB')
plt.ylabel('pixel count')
plt.xlim(-10, 10)
plt.grid(True, alpha=0.3)
plt.show()


## From ratio_db to a binary change mask
The script uses `abs(ratio_db) > change_thr_db`.
Try different thresholds and compare how much area gets flagged.


In [ ]:
def pct_flagged(arr, thr):
    m = (np.abs(arr) > thr)
    return 100.0 * np.mean(m[np.isfinite(arr)])

for thr in [1.0, 1.5, 2.0, 3.0, 4.0]:
    print(f'thr={thr:.1f} dB -> {pct_flagged(ratio_db, thr):.2f}% flagged')

thr = 2.0
mask = (np.abs(ratio_db) > thr).astype('uint8')
plt.figure(figsize=(7,6))
plt.imshow(mask, vmin=0, vmax=1)
plt.title(f'Change mask (abs(ratio_db) > {thr} dB)')
plt.colorbar(label='0/1')
plt.axis('off')
plt.show()


## Next steps (if you want to harden this)
- Speckle reduction (e.g., Lee filter or multi-temporal median)
- Coherence-based change (needs SLC)
- Add landcover priors or water occurrence layers
- Validate against ground truth or authoritative flood extents
